# Autogen with GPT-4o

## ConversableAgent

In [1]:
import os
from autogen import ConversableAgent, AssistantAgent, GroupChat, GroupChatManager
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
key = os.environ["AZURE_OPENAI_KEY"]
api = os.environ["API_VERSION"]
deployment = os.environ["CHAT_COMPLETIONS_DEPLOYMENT_NAME"]


### Automatic Cache feature

You will need to delete the .cache folder or set a new cache_seed
[LLM Caching with Autogen](https://microsoft.github.io/autogen/docs/topics/llm-caching/#:~:text=To%20change%20the%20cache%20directory%2C%20set%20cache_path_root%3A%20with,on%20by%20default%20with%20cache_seed%20set%20to%2041.)

In [2]:
# This has a cache_seed
llm_config = {"model": deployment, "base_url": endpoint, "api_key": key, "api_type": "azure", "api_version": api,
               "temperature": 0.8, "max_tokens": 4096, "top_p": 0.95, "cache_seed": 13}

# THIS HAS NO CACHE SEED SET
# llm_config = {"model": deployment, "base_url": endpoint, "api_key": key, "api_type": "azure", "api_version": api,
#                "temperature": 0.8, "max_tokens": 4096, "top_p": 0.95, "cache_seed": None}

baseline = ConversableAgent(
    name="Baseline Comparison Bot",
    system_message="You are an AI assistant that helps people find information.",
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input. human_input_mode: Literal['ALWAYS', 'NEVER', 'TERMINATE'] = "TERMINATE",
)

In [3]:
baselinereply = baseline.generate_reply(messages=[{"content": "Write a persuasive essay about why Dr. Dog is the greatest band ever. Use markdown format. The essay should start with an outline and aim to be close to 2,000 words.",
                                              "role": "user"}])
print(baselinereply)


# Why Dr. Dog is the Greatest Band Ever

## Outline

1. **Introduction**
    - Hook: Capturing the reader's attention with a strong statement or quote.
    - Thesis statement: Dr. Dog's unique blend of musical innovation, lyrical depth, and enduring fan engagement make them the greatest band ever.

2. **Musical Innovation**
    - Varied Influences: How Dr. Dog blends genres.
    - Evolution of Sound: Discussing their discography and how their sound has evolved.
    - Technical Proficiency: Highlighting their instrumental and vocal skills.

3. **Lyrical Depth**
    - Poetic Lyrics: Analysis of key songs.
    - Themes: Exploration of recurring themes in their music.
    - Storytelling: How their lyrics create vivid narratives.

4. **Enduring Fan Engagement**
    - Community Building: How the band interacts with their fans.
    - Live Performances: The energy and intimacy of their live shows.
    - Longevity: How they have maintained a loyal fan base over the years.

5. **Critical Acclaim

## Set Up Multi-Agent Conversation

> NOTE: the name= parameter must not include spaces

In [4]:
# Agent for collecting a topic from a user
topic_agent = ConversableAgent(
    name="TopicCollectionAgent",
    system_message="You are part of a persuasive essay writing team and you present the topic for the essay to the group. ",
    llm_config=llm_config,
    description="Collect a topic from a user for a persuasive essay.",
    human_input_mode="NEVER",
)


In [5]:
# Agent for creating an outline
outline_agent = ConversableAgent(
    name="OutlineAgent",
    system_message="You are part of a persuasive essay writing team and you create a detailed outline for the persuasive essay based on the supplied topic. ",
    llm_config=llm_config,
    description="Create an outline for a persuasive essay based on a topic.",
    human_input_mode="NEVER",
)

In [6]:
# Agent for creating an outline
writing_agent = ConversableAgent(
    name="WritingAgent",
    system_message="You are part of a persuasive essay writing team and you act as the main author. You should begin writing the essay based on the supplied outline. ",
    llm_config=llm_config,
    description="Write a persuasive essay based on a supplied outline.",
    human_input_mode="NEVER",
)

In [7]:
# Agent for creating an outline
review_agent = ConversableAgent(
    name="ReviewAgent",
    system_message="You are part of a persuasive essay writing team and you act as the reviewer and critic. Keep a keen eye on the structure, grammar, and overall persuasiveness of the essay. Return the final product in markdown format.",
    llm_config=llm_config,
    description="Review a persuasive essay and make necessary edits.",
    human_input_mode="NEVER",
)

### Using GroupChat

In [8]:
# Create the GroupChat object
writing_group_chat = GroupChat(
    agents=[topic_agent, outline_agent, writing_agent, review_agent],
    messages=[],
    max_round=10, # 10 is default
    send_introductions=True, # send_introductions: send a round of introductions at the start of the group chat,
     # so agents know who they can speak to (default: False)
    speaker_selection_method='auto', # 'auto' is default ['auto', 'manual', 'random', 'round_robin']
)

In [9]:
# Create the GroupChatManager object
writing_group_manager = GroupChatManager(
    groupchat=writing_group_chat,
    llm_config=llm_config,
)

In [10]:
chat_result = topic_agent.initiate_chat(
    writing_group_manager,
    message="Why Dr. Dog is the greatest band ever.",
    summary_method="reflection_with_llm",
)

TopicCollectionAgent (to chat_manager):

Why Dr. Dog is the greatest band ever.

--------------------------------------------------------------------------------
Why Dr. Dog is the greatest band ever.

--------------------------------------------------------------------------------
OutlineAgent (to chat_manager):

Thank you for the topic! We'll now proceed to create a detailed outline for the persuasive essay on "Why Dr. Dog is the greatest band ever."

OutlineAgent, please take over from here.

--------------------------------------------------------------------------------
OutlineAgent (to chat_manager):

Absolutely, here is a detailed outline for the persuasive essay on "Why Dr. Dog is the greatest band ever":

### I. Introduction
   - **A. Hook:** Capture the reader's attention with an interesting fact or quote about Dr. Dog.
   - **B. Background Information:** Briefly introduce Dr. Dog, including the band's origin, genre, and key members.
   - **C. Thesis Statement:** Clearly stat